In [51]:
import importlib
import FuncionesLectura as fl

In [52]:
importlib.reload(fl)

df_corpus=fl.leerCorpus(ruta="data/transcripciones", limpiar=False)
df_corpus.head()

,id,speaker,transcription,text,gloss_es,free_translation,morpheme_break,pos,file
0,002,Roberto,entonces estamos grabando estamos acá con Elias,entonces estamos grabando estamos acá con Elias,,entonces estamos grabando estamos acá con Elias,,,EC-cancion.aku-2013.txt
1,003,Roberto,y Elias va a hacer una canción propia de iscon...,y Elias va a hacer una canción propia de iscon...,,y Elias va a hacer una canción propia de iscon...,,,EC-cancion.aku-2013.txt
2,004,Roberto,ya puedes empezar,ya puedes empezar,,ya puedes empezar,,,EC-cancion.aku-2013.txt
3,005,Elias,non kentihokobi,non kentihokobi,1PL =GEN olla -DIM =ENF,nuestra ollita,no =n kenti -hoko =bi,pron =clit. n. -suf =clit.,EC-cancion.aku-2013.txt
4,006,Elias,non kentihokobi,non kentihokobi,1PL =GEN olla -DIM =ENF,nuestra ollita,no =n kenti -hoko =bi,pron =clit. n. -suf =clit.,EC-cancion.aku-2013.txt


In [53]:
file_path = 'data/diccionario.txt'
df = fl.parse_txt(file_path)
df

,lex_isc,lex_citation,pos,gloss_es,def_es
0,a,NaN,pron.,él ; ella,"pronombre de tercera persona singular: él, ella."
1,abebi choroin,NaN,v.intr.,correr (con otro),"correr con otro, como en una carrera."
2,aben,NaN,n.,mono capuchino (grande) o mono martí,variedad más grande de mono capuchino o martín...
3,aben ino,NaN,n.,puma o puma,felino de gran tamaño y un solo color sin manc...
4,abo,NaN,pron.,ellos ; ellas,"pronombre de tercera persona plural: ellos, el..."
...,...,...,...,...,...
1709,yunpa,NaN,n.,iguana,lagartija muy grande. Anoles Fam. Polychrotidae
1710,yunpa,NaN,n.,iguana,iguana de gran tamaño y de color verde.
1711,yunsha,NaN,n.,shuyo,Pez similar al huasaco pero de menor tamaño y ...
1712,yushin meshti,NaN,n.,mántido o manta,"insecto de cuerpo delgado, de color verde y pa..."
